In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline

np.random.seed(seed=42)

In [11]:
import functions.CreatingNewDataframe as CreatingNewDataframe
import functions.GenerateFis as Features
import functions.GenerateMeansWithOffset as MeansOffset


from functions.date_split import split_month_test
from score_submission import our_scorer, scorer


In [3]:
from sklearn import linear_model
from sklearn.base import BaseEstimator, TransformerMixin


In [4]:
df_initial = pd.read_csv('train.csv.zip', parse_dates=['DATE'])
ids = np.unique(df_initial.ATM_ID)
np.random.shuffle(ids)

In [5]:
# means = MeansOffset.CreateColumnsOfMeans(df_initial, window=7)


In [6]:
class SeparatingAtmIds(BaseEstimator, TransformerMixin):
    """Extract the subject & body from a usenet post in a single pass.

    Takes a sequence of strings and produces a dict of sequences.  Keys are
    `subject` and `body`.
    """
    def fit(self, x, y=None):
        self.train = True
        return self

    def transform(self, X):
        if self.train:
            print("TRAIN ONLY CALLED")
            print("Separating finished")
            return CreatingNewDataframe.CleanDataFrame_TRAIN_ONLY(df=X, window_size=15)
        else:
            print("TEST ONLY CALLED")
            print("Separating finished")
            return CreatingNewDataframe.CleanDataFrame_TEST(df=X, window_size=15)

In [7]:
class FinalStepForSubmission(BaseEstimator, TransformerMixin):
    """
    FINAL
    """
    
    def fit(self, X, y=None):
        return self

    def predict(self, X):
        X = X[['DATE', "ATM_ID", "y_predict"]]
        X.columns = ['DATE', "ATM_ID", "CLIENT_OUT"]
        print(len(np.unique(X.ATM_ID)))
        print("FinalStepForSubmission Finished")
        return X
        
class WrapperForEstimator(BaseEstimator, TransformerMixin):
    """
    FINAL
    """
    def __init__(self, estimator):
        self.estimator = estimator
    
    
    def fit(self, X, y=None):
        indices_without_nan = X.isnull().any(axis=1)
        indices_without_nan = X.index[indices_without_nan]
        
        X.drop(axis=0, index=indices_without_nan, inplace=True)
        y.drop(axis=0, index=indices_without_nan, inplace=True)
        
        print("WrapperForEstimator fit called")
        print(X.columns)
        
        self.estimator.fit(X.drop(['DATE',  'ATM_ID', 'CLIENT_OUT'], axis=1, inplace=False),
                           y)
        
        return self

    def transform(self, X):
        print("WrapperForEstimator transform called")
        
        y_pred = self.estimator.predict(X.drop(['DATE',  'ATM_ID', 'CLIENT_OUT'], axis=1, inplace=False))
        
        X_tr = X.copy()
        X_tr['y_predict'] = y_pred
        print("Transform from estimator Finished")
        return X_tr

In [8]:
from sklearn.ensemble import RandomForestRegressor

In [9]:
X = df_initial
y = df_initial[['CLIENT_OUT']]

In [10]:
pipeline.fit(X, y)

NameError: name 'pipeline' is not defined

In [ ]:
pipeline.predict(X)

In [ ]:
# The dates to predict
pred_dates  = ['2017-08-16', '2017-08-17', '2017-08-18', '2017-08-19',
               '2017-08-20', '2017-08-21', '2017-08-22', '2017-08-23',
               '2017-08-24', '2017-08-25', '2017-08-26', '2017-08-27',
               '2017-08-28', '2017-08-29', '2017-08-30', '2017-08-31',
               '2017-09-01', '2017-09-02', '2017-09-03', '2017-09-04',
               '2017-09-05', '2017-09-06', '2017-09-07', '2017-09-08',
               '2017-09-09', '2017-09-10', '2017-09-11', '2017-09-12',
               '2017-09-13', '2017-09-14', '2017-09-15', '2017-09-16',
               '2017-09-17']

rows = []



for id in ids:
    mean = 0
#     df[df.ATM_ID==id].CLIENT_OUT.mean()
    for date in pred_dates:
        rows.append((date, id, mean))
submission = pd.DataFrame(rows, columns = ['DATE', 'ATM_ID', 'CLIENT_OUT'])
# submission.to_csv('submission.csv', index=False)



In [ ]:
submission = pd.DataFrame(rows, columns = ['DATE', 'ATM_ID', 'CLIENT_OUT'])


In [ ]:
# current_scores = cross_val_score(pipeline, X, y, scoring=our_scorer,  groups=X['ATM_ID'].values, cv=5)

In [ ]:
current_scores

In [ ]:
def FlattenList(l_of_l):
    return np.array([x for sublist in l_of_l for x in sublist])

In [ ]:
FlattenList([[x, x+1, x+2, x+3, x+4, x+5, x+6] for x in range(0, 100, 33)])

In [ ]:
train_df, test_df = split_month_test(df_initial,
                                    test_days=33,
                                     train_days=0,
                                     unuseddays_in_end=0)
        

In [ ]:
pipeline = Pipeline([
    ("rfc", WrapperForEstimator(RandomForestRegressor(n_jobs=-1))), #predict step
    ("submission", FinalStepForSubmission()),
])

In [ ]:
def CalculateScore(pipelines, df=df_initial):
    test_days = 33
    
    scores = []
    
    for unuseddays_in_end in range(0, 28, 28):
        train_df, test_df = split_month_test(df,
                                             test_days=test_days,
                                             train_days=0,
                                             unuseddays_in_end=unuseddays_in_end)
        
        y_test = test_df['CLIENT_OUT']
        test_df['CLIENT_OUT'] = None
        
        DF_FOR_FEATURES = pd.merge([train_df, test_df], axis=0)
        
        df1, df2, df3, df4, df5 = AddFeatures(DF_FOR_FEATURES)


        end_of_train = train_df.shape[0]
        
        
        df_train_1 = df1[:end_of_train]
        df_train_2 = df2[:end_of_train]
        df_train_3 = df3[:end_of_train]
        df_train_4 = df4[:end_of_train]
        df_train_5 = df5[:end_of_train]
        df_trains = [df_train_1, df_train_2, df_train_3, df_train_4, df_train_5]
        
        indices = FlattenList([[x, x+1, x+2, x+3, x+4, x+5, x+6] for x
                            in range(0, test_df.shape[0], test_days)]) + end_of_train
        df_test_1 = df1[indices]
        y_test_1 = y_test[indices]
        
        indices = FlattenList([[x, x+1, x+2, x+3, x+4, x+5, x+6] for x
                            in range(7, test_df.shape[0], test_days)]) + end_of_train
        
        df_test_2 = df2[indices]
        y_test_2 = y_test[indices]
        
        indices = FlattenList([[x, x+1, x+2, x+3, x+4, x+5, x+6] for x
                            in range(14, test_df.shape[0], test_days)]) + end_of_train
        
        df_test_3 = df3[indices]
        y_test_3 = y_test[indices]        
        
        indices = FlattenList([[x, x+1, x+2, x+3, x+4, x+5, x+6] for x
                            in range(21, test_df.shape[0], test_days)]) + end_of_train
        
        df_test_4 = df4[indices]
        y_test_4 = y_test[indices]        
        
        indices = FlattenList([[x, x+1, x+2, x+3, x+4, ] for x
                            in range(28, test_df.shape[0], test_days)]) + end_of_train
        
        df_test_5 = df5[indices]
        y_test_5 = y_test[indices]
        
        
        df_tests = [df_test_1, df_test_2, df_test_3, df_test_4, df_test_5]
        y_tests = [y_test_1, y_test_2, y_test_3, y_test_4, y_test_5]
    
        
        y_preds = []
        for i in range(5):
            pipelines[i].fit(df_trains[i], PUT Y HERE)
            y_pred = models[i].predict(df_tests[i])            
            y_preds.append(y_pred)
            
        
        
    return scores

In [ ]:
np.random.shuffle(ids)

In [ ]:
ids = np.array([283, 371, 209,  10, 319, 311, 276, 138,   6, 245, 327,  69, 302,
        55, 257, 305, 403, 291, 203, 231,  92, 142, 343, 345, 127,  52,
       389, 413, 217, 328,  23, 230,  41,  30, 299, 154,   8, 105,  56,
        87, 108,  90, 396,  72,  91,  76, 321, 315, 140, 317, 255, 194,
       330,  89, 415, 385, 113, 107, 272,  31, 318,  71,   7, 262, 243,
        36,  28, 357,  68, 264, 136, 167, 316, 273, 172, 390,  96,  98,
        79, 401, 228, 212,  25, 163,  93, 147, 151,  86,  21,  11, 274,
       119,  81, 309,  47,  22, 252, 199, 380,  80, 314,  82,  38, 258,
       370, 310, 189,  24, 256, 112, 177, 367, 100, 342, 339, 149, 196,
       406, 277, 286,  67, 242, 260, 235, 186, 352,  48, 166, 275, 391,
       182, 141, 238, 150, 383, 376, 213, 246, 249,   3, 148, 193, 248,
       155, 290, 362, 168,  88,  35,  97, 374, 131, 307, 219, 387, 340,
       306,  78, 278,  99, 297, 220,  17,  44,  34, 192, 394, 354, 240,
        63, 111, 288, 341,  51, 103, 338, 181,  32, 407, 191, 280,   0,
       373, 378, 117, 361, 355, 241, 115,  45,  73, 207, 326, 282, 336,
        14, 414, 372,  33, 303,   5, 160,  40, 284, 226, 289, 395, 185,
        74, 180, 169, 404,   9,  84, 369,  77,  54, 386, 216,  50, 161,
       381, 300, 211,  29, 366, 139,  95, 287, 295,  49, 331, 405, 360,
       279,  19,  57, 109, 368, 312, 379, 281, 225,  53, 301, 195, 270,
       304, 347,   4, 128,  65, 178,   2, 176, 126,  60,  94, 285,  43,
       106,  64, 334,  18, 102, 233, 408, 179, 399, 337,  66,  61, 244,
       269, 418, 265, 239,  70,  58, 324, 348,  27, 222, 382, 271, 359,
       205, 173, 208, 116, 101, 298, 159, 419,  26, 267,  85, 129, 375,
       123])

In [ ]:

#         train_df1, test_df1 = split_month_test(df1,
#                                      test_days=test_days,
#                                      train_days=0,
#                                      unuseddays_in_end=unuseddays_in_end)
        
#         train_df2, test_df2 = split_month_test(df2,
#                                      test_days=test_days,
#                                      train_days=0,
#                                      unuseddays_in_end=unuseddays_in_end)

#         train_df3, test_df3 = split_month_test(df3,
#                                      test_days=test_days,
#                                      train_days=0,
#                                      unuseddays_in_end=unuseddays_in_end)

#         train_df4, test_df4 = split_month_test(df4,
#                                      test_days=test_days,
#                                      train_days=0,
#                                      unuseddays_in_end=unuseddays_in_end)

#         train_df5, test_df5 = split_month_test(df5,
#                                      test_days=test_days,
#                                      train_days=0,
#                                      unuseddays_in_end=unuseddays_in_end)

In [ ]:
ids

In [ ]:
ids[]